In [1]:
import pandas as pd
import numpy as np
import psycopg2
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec
from typing import List
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java

ZEMBEREK_PATH = r'C:\Users\IRPHAN\Desktop\zemberek-full.jar'
startJVM(getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\IRPHAN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\IRPHAN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\IRPHAN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\IRPHAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
connection = psycopg2.connect(user = "postgres",
                                  password = "42088",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="postgres")                           
cursor = connection.cursor()
cursor2 = connection.cursor()
cursor3 = connection.cursor()
postgreSQL_select_Query = "select * from public.train" #our trained data
postgreSQL_select_Query2 = "select * from public.data_test" #our test data
postgreSQL_select_Query3 = "select * from public.trained_data" #our comp data
cursor.execute(postgreSQL_select_Query)
cursor2.execute(postgreSQL_select_Query2)   
cursor3.execute(postgreSQL_select_Query3)   
records = cursor.fetchall()
records2 = cursor2.fetchall()
records3 = cursor3.fetchall()

In [3]:
df_comp = pd.DataFrame(records3, columns=["id", "metin","network_ici",
                    "network_disi",
                    "genisbant",
                    "gb_kopma",
                    "gb_altyapi",
                    "gb_port",
                    "gb_kablo",
                    "gb_hizveping",
                    "gb_modem",
                    "sabithat",
                    "sh_kopma",
                    "sh_altyapi",
                    "mobil",
                    "mb_baglanti",
                    "mb_hizveveri",
                    "mb_kopma",
                    "tivibu",
                    "tv_cihaz",
                    "tv_yayin",
                    "siparis_nakil",
                    "ekip",
                    "bayi",
                    "diger"])
df_train = pd.DataFrame(records, columns=["id", "metin","network_ici",
                    "network_disi",
                    "genisbant",
                    "gb_kopma",
                    "gb_altyapi",
                    "gb_port",
                    "gb_kablo",
                    "gb_hizveping",
                    "gb_modem",
                    "sabithat",
                    "sh_kopma",
                    "sh_altyapi",
                    "mobil",
                    "mb_baglanti",
                    "mb_hizveveri",
                    "mb_kopma",
                    "tivibu",
                    "tv_cihaz",
                    "tv_yayin",
                    "siparis_nakil",
                    "ekip",
                    "bayi",
                    "diger"])
df_test = pd.DataFrame(records2, columns=["id", "metin","network_ici",
                    "network_disi",
                    "genisbant",
                    "gb_kopma",
                    "gb_altyapi",
                    "gb_port",
                    "gb_kablo",
                    "gb_hizveping",
                    "gb_modem",
                    "sabithat",
                    "sh_kopma",
                    "sh_altyapi",
                    "mobil",
                    "mb_baglanti",
                    "mb_hizveveri",
                    "mb_kopma",
                    "tivibu",
                    "tv_cihaz",
                    "tv_yayin",
                    "siparis_nakil",
                    "ekip",
                    "bayi",
                    "diger"])

In [4]:
converted_list = []
for satir in open('C:\\Users\\IRPHAN\\Desktop\\New folder (6)\\stop_words_turkish.txt',encoding="utf-8"):#https://countwordsfree.com/stopwords/turkish
    converted_list.append(satir.strip())

In [5]:
TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
morphology = TurkishMorphology.createWithDefaults()

In [6]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in converted_list]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# Tokenize the sentence
def lemmatizer(string):
    analysis: java.util.ArrayList = (
        morphology.analyzeAndDisambiguate(string).bestAnalysis()
        )
    pos: List[str] = []
    for i, analysis in enumerate(analysis, start=1):
        f'\nAnalysis {i}: {analysis}',
        f'\nPrimary POS {i}: {analysis.getPos()}'
        f'\nPrimary POS (Short Form) {i}: {analysis.getPos().shortForm}'

        pos.append(
            f'{str(analysis.getLemmas()[0])}'
            )
    return " ".join(pos)

In [7]:
#Pre-processing the dataset
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
df_train['clean_text'] = df_train['metin'].apply(lambda x: finalpreprocess(x))
df_train.head(5)

,id,metin,network_ici,network_disi,genisbant,gb_kopma,gb_altyapi,gb_port,gb_kablo,gb_hizveping,...,mb_hizveveri,mb_kopma,tivibu,tv_cihaz,tv_yayin,siparis_nakil,ekip,bayi,diger,clean_text
0,17237993,TTNET ev internetimin bina girişinden kablo ba...,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,ttnet ev internet bina gir kablo bağlantı yer ...
1,17237543,06/Ocak/2020 den beri sürekli arızalı olan ve ...,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,ocak de sürekli arıza teknik ekip ziyaret türl...
2,17232767,"100 lira fatura veriyorum, hala internetim yav...",True,False,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,lira fatura ver hala internet yavaş internet o...
3,17228655,Aylardır kullanıyoruz Türk Telekom ev internet...,True,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,ay kullan türk telekom ev internet son ay git ...
4,17227857,23.11.2020 tarihinde İstanbul Sancaktepe buluş...,True,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,tarih istanbul sancaktepe buluş sokak UNK site...


In [8]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(df_train["clean_text"],df_train["network_ici"],test_size=0.2,shuffle=True)
#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  
X_test_tok= [nltk.word_tokenize(i) for i in X_test]


In [9]:
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

#building Word2Vec model
"""
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
            return self
    def transform(self, X):
            return np.array([
                np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                        or [np.zeros(self.dim)], axis=0)
                for words in X
            ])
  

w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))
df_train['clean_text_tok']=[nltk.word_tokenize(i) for i in df_train['clean_text']]
model = Word2Vec(df_train['clean_text_tok'],min_count=1)     
modelw = MeanEmbeddingVectorizer(w2v)
# converting text to numerical data using Word2Vec
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_test_tok)
"""

"\nclass MeanEmbeddingVectorizer(object):\n    def __init__(self, word2vec):\n        self.word2vec = word2vec\n        # if a text is empty we should return a vector of zeros\n        # with the same dimensionality as all the other vectors\n        self.dim = len(next(iter(word2vec.values())))\n    def fit(self, X, y):\n            return self\n    def transform(self, X):\n            return np.array([\n                np.mean([self.word2vec[w] for w in words if w in self.word2vec]\n                        or [np.zeros(self.dim)], axis=0)\n                for words in X\n            ])\n  \n\nw2v = dict(zip(model.wv.index_to_key, model.wv.vectors))\ndf_train['clean_text_tok']=[nltk.word_tokenize(i) for i in df_train['clean_text']]\nmodel = Word2Vec(df_train['clean_text_tok'],min_count=1)     \nmodelw = MeanEmbeddingVectorizer(w2v)\n# converting text to numerical data using Word2Vec\nX_train_vectors_w2v = modelw.transform(X_train_tok)\nX_val_vectors_w2v = modelw.transform(X_test_tok)\n

In [17]:
print(X_train_vectors_tfidf[0])

  (0, 1620)	0.24142310507926085
  (0, 2292)	0.14238226002686805
  (0, 734)	0.20225681845057428
  (0, 3293)	0.1645767405970466
  (0, 3161)	0.16154358429175542
  (0, 1987)	0.19216731395177566
  (0, 1476)	0.10400698312193196
  (0, 3373)	0.1565599947163078
  (0, 3402)	0.1269246210506153
  (0, 140)	0.1479123278603232
  (0, 2229)	0.16593733688485932
  (0, 1657)	0.14919607052020742
  (0, 208)	0.16196540334121134
  (0, 2490)	0.2902813797721236
  (0, 2228)	0.31298160291271837
  (0, 2604)	0.18382564054170428
  (0, 691)	0.22332617054007642
  (0, 1021)	0.18586670371880953
  (0, 674)	0.09298776123896713
  (0, 2089)	0.3605879665688933
  (0, 89)	0.19888862550143838
  (0, 1207)	0.11181505228571564
  (0, 1447)	0.07101027833847182
  (0, 2200)	0.14919607052020742
  (0, 1338)	0.2874335216865927
  (0, 654)	0.16616787657871543
  (0, 2829)	0.07766714891189974
  (0, 2935)	0.07970661578717682


In [12]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]

print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

       False       0.89      0.93      0.91       318
        True       0.91      0.87      0.89       282

    accuracy                           0.90       600
   macro avg       0.90      0.90      0.90       600
weighted avg       0.90      0.90      0.90       600

Confusion Matrix: [[295  23]
 [ 37 245]]
AUC: 0.9643605870020965


In [51]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression (W2v)
lr_w2v=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_w2v.fit(X_train_vectors_w2v, y_train)  #model
#Predict y value for test dataset
y_predict = lr_w2v.predict(X_val_vectors_w2v)
y_prob = lr_w2v.predict_proba(X_val_vectors_w2v)[:,1]

print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

       False       0.87      0.86      0.86       340
        True       0.82      0.83      0.83       260

    accuracy                           0.85       600
   macro avg       0.84      0.85      0.84       600
weighted avg       0.85      0.85      0.85       600

Confusion Matrix: [[291  49]
 [ 43 217]]
AUC: 0.9255769230769231


In [68]:
X_train_vectors_w2v.shape

(2398, 100)

In [52]:
#FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = nb_tfidf.predict(X_test_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]

print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

       False       0.87      0.90      0.88       340
        True       0.86      0.83      0.84       260

    accuracy                           0.87       600
   macro avg       0.87      0.86      0.86       600
weighted avg       0.87      0.87      0.87       600

Confusion Matrix: [[305  35]
 [ 45 215]]
AUC: 0.9354524886877829


In [13]:
#Pre-processing the new dataset
df_test['clean_text'] = df_test['metin'].apply(lambda x: finalpreprocess(x)) #preprocess the data
X_test=df_test['clean_text'] 

In [14]:
#converting words to numerical data using tf-idf
X_vector=tfidf_vectorizer.transform(X_test)
#use the best model to predict 'target' value for the new dataset 
y_predict = lr_tfidf.predict(X_vector)      
y_prob = lr_tfidf.predict_proba(X_vector)[:,1]
df_test['predict_prob']= y_prob
df_test['network_ici']= y_predict
final=df_test[['id','clean_text','network_ici']].reset_index(drop=True)
print(final.head())

         id                                         clean_text  network_ici
0  15500032  hat değiş önce telekom numara değiş prime müşt...        False
1  14968518  no internet hat nakil al iste altyapı nakil ya...         True
2  14495278  iş sebep sabit adres neredeyse yıl internet ku...        False
3  12254763  dün türk telekom jet mobile internet fatura ya...        False
4  17410879  fatura tarife yüksek gel yap öde sınır aşım du...        False


In [15]:
#we compare the test results with exact values
counter = 0
i = 0
j = 0
for row in final["id"]:
    #print(row)
    for row2 in df_comp["id"]:
        if(final["id"][i] == df_comp["id"][j]):
            #print(i)
            if(final["network_ici"][i] == df_comp["network_ici"][j]):
                counter +=1      
        j+=1
    i += 1
    j = 0
    #print("counter = {}".format(counter))
print("Accuracy = %{}".format(round((counter/i)*100,2)))

Accuracy = %88.08


In [58]:
#converting words to numerical data using word2vec
X_vector=modelw.transform(X_test)
#use the best model to predict 'target' value for the new dataset 
y_predict = lr_w2v.predict(X_vector)      
y_prob = lr_w2v.predict_proba(X_vector)[:,1]
df_test['predict_prob']= y_prob
df_test['network_ici']= y_predict
final=df_test[['id','clean_text','network_ici']].reset_index(drop=True)
print(final.head())

         id                                         clean_text  network_ici
0  15500032  hat değiş önce telekom numara değiş prime müşt...        False
1  14968518  no internet hat nakil al iste altyapı nakil ya...        False
2  14495278  iş sebep sabit adres neredeyse yıl internet ku...        False
3  12254763  dün türk telekom jet mobile internet fatura ya...        False
4  17410879  fatura tarife yüksek gel yap öde sınır aşım du...        False


In [59]:
#we compare the test results with exact values
counter = 0
i = 0
j = 0
for row in final["id"]:
    #print(row)
    for row2 in df_comp["id"]:
        if(final["id"][i] == df_comp["id"][j]):
            #print(i)
            if(final["network_ici"][i] == df_comp["network_ici"][j]):
                counter +=1      
        j+=1
    i += 1
    j = 0
    #print("counter = {}".format(counter))
print("Accuracy = %{}".format(round((counter/i)*100,2)))

Accuracy = %60.59


In [60]:
#converting words to numerical data using Naive Bayes(tf-idf)
X_vector=tfidf_vectorizer.transform(X_test)
#use the best model to predict 'target' value for the new dataset 
y_predict = nb_tfidf.predict(X_vector)      
y_prob = nb_tfidf.predict_proba(X_vector)[:,1]
df_test['predict_prob']= y_prob
df_test['network_ici']= y_predict
final=df_test[['id','clean_text','network_ici']].reset_index(drop=True)
print(final.head())

         id                                         clean_text  network_ici
0  15500032  hat değiş önce telekom numara değiş prime müşt...        False
1  14968518  no internet hat nakil al iste altyapı nakil ya...         True
2  14495278  iş sebep sabit adres neredeyse yıl internet ku...        False
3  12254763  dün türk telekom jet mobile internet fatura ya...        False
4  17410879  fatura tarife yüksek gel yap öde sınır aşım du...        False


In [61]:
#we compare the test results with exact values
counter = 0
i = 0
j = 0
for row in final["id"]:
    #print(row)
    for row2 in df_comp["id"]:
        if(final["id"][i] == df_comp["id"][j]):
            #print(i)
            if(final["network_ici"][i] == df_comp["network_ici"][j]):
                counter +=1      
        j+=1
    i += 1
    j = 0
    #print("counter = {}".format(counter))
print("Accuracy = %{}".format(round((counter/i)*100,2)))

Accuracy = %87.88


In [ ]:
#NLP Tutorial for Text Classification in Python (with Twitter datas)
#https://medium.com/analytics-vidhya/nlp-tutorial-for-text-classification-in-python-8f19cd17b49e 